In [36]:
from __future__ import division
import os
import glob

import pandas as pd
import numpy as np

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\manon\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Global variable definition 

In [27]:
file_directory = '../Untitled folder/allconversations/'
text_files_sat = glob.glob('../Untitled folder/allconversations/sat/*.txt')
text_files_dsat = glob.glob ('../Untitled folder/allconversations/dsat/*.txt')

In [17]:
# sentences to delete
dic = {'thankyou':"", 'thank you':"", "goodbye":"", "good bye":"", 'bye':""}

In [22]:
def number_repetition(file):
    with open (file) as f:
        lines = f.readlines()
        new_list =[]
        for line in lines:
            if "SYSTEM" in line:
                new_list.append(line.strip())
    #create dictionary and count the repetition of sentence (Feature 1)
    new_dict = {}
    for li in new_list:
        if li in new_dict:
            new_dict[li] += 1
        else:
            new_dict[li] = 1
    num_rep = sum([value for value in new_dict.values() if value > 1])
    
    #create a matrix to calculate repetition (Feature 2)
    m = [[0]* len(new_list) for i in range(len(new_list))]
    for i, x in enumerate(new_list):
        for j, y in enumerate(new_list):
            if x == y:
                m[i][j] = 1 
    all_value = 0
    for li in m:
        a = sum(li)
        all_value+=a
    num_rep_exp = (all_value-len(new_list))/2
    
    #Percentage of the repetition sentence in the entire conversation (Feature 3)
    num_rep_per = num_rep/len(lines)
    #Number of sentences in the conversation (Feature 4)
    len_conversation = len(lines)
    
    return num_rep, num_rep_per,num_rep_exp,len_conversation

### Function definition

In [64]:
def pos_neg_conv(file):
    # get everything
    
    dict_conv_only = create_user_conv(file)
    compound_score, tot_pos_sen, tot_neg_sen = get_sentiment(dict_conv_only)
    return compound_score, tot_pos_sen, tot_neg_sen
    
    
def get_sentiment(dict_conv_only):
    # generate the compound sentiment score and store in conversation

    for sentence in dict_conv_only.keys():
        compound_sentence = generatesentiment(sentence)
        # add compound score to sentence 
        dict_conv_only[sentence] = compound_sentence
    # add total compound score of sentence to conversation
    compound_score = sum(dict_conv_only.values())/len(dict_conv_only.values())
    tot_pos_sen = len([x for x in dict_conv_only.values() if x > 0])/len(dict_conv_only.values())
    tot_neg_sen = len([x for x in dict_conv_only.values() if x < 0])/len(dict_conv_only.values())
    return compound_score, tot_pos_sen, tot_neg_sen
    
    
def create_user_conv(file):
    #create a dict per conversation of only USER sentences 
    # and 'thankyou goodbye left out'
    conv_only_user = {}
    with open(file, 'r') as f:
        data =f.readlines()
        new_list =[]
        for line in data:
            # only USER input
            if "USER" in line:
                # Remove the word '[USER]'
                line = line.replace("[USER]", "")
                line = line.strip()
                new_list.append(line)
        #strip to get the conversation id 
        file = file.replace('../Untitled folder/allconversations/sat\\', "")   
        file = file.replace('.txt', "") 
        # Remove 'thank you good bye' if in last sentences
        for i,j in dic.items():
            new_list[-1] = new_list[-1].replace(i, j)
            new_list[-2] = new_list[-2].replace(i, j)
        conv_only_user[file] = new_list
        
        # convert to dict 
        # with every conversation a dict with sentences as keys
        # and compound pos neg score (as 0)
        for conv in conv_only_user.values():
            new_dict2 = {}
            for sentence in conv:
                new_dict2[sentence] = 0
        return new_dict2       
        
def generatesentiment(sentence):
    # function for sentiment analysis on sentences
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sentence)
    return ss['compound']


   

In [65]:
columns = ['num_rep','num_rep_per','num_rep_exp','len_conversation', 'total_compound_conv', 'tot_pos_sen', 'tot_neg_sen', 'Is_satisfied']
df_rep_dsat = pd.DataFrame(index=range(len(text_files_dsat)), columns=columns)

In [66]:
for idx,file in enumerate(text_files_dsat):
    num_rep, num_rep_per,num_rep_exp,len_conversation = number_repetition(file)
    compound_score, tot_pos_sen, tot_neg_sen = pos_neg_conv(file)
    df_rep_dsat.iloc[idx] = pd.Series({'num_rep':num_rep, 'num_rep_per':num_rep_per,
                                    'num_rep_exp':num_rep_exp,'len_conversation':len_conversation,
                                       'total_compound_conv':compound_score, 'tot_pos_sen':tot_pos_sen, 
                                       'tot_neg_sen':tot_neg_sen, 'Is_satisfied':0})

In [67]:
df_rep_dsat

,num_rep,num_rep_per,num_rep_exp,len_conversation,total_compound_conv,tot_pos_sen,tot_neg_sen,Is_satisfied
0,7,0.291667,21,24,0,0,0,0
1,4,0.333333,2,12,0.10275,0.333333,0.166667,0
2,4,0.285714,2,14,0.0463857,0.142857,0.142857,0
3,0,0,0,10,0,0,0,0
4,4,0.25,2,16,0,0,0,0
5,0,0,0,6,0,0,0,0
6,12,0.333333,46,36,0.0140846,0.153846,0.0769231,0
7,9,0.375,36,24,-0.02044,0.2,0.2,0
8,13,0.270833,20,48,0.187467,0.416667,0.0416667,0
9,18,0.321429,22,56,-0.016908,0.2,0.16,0


In [ ]:



#create a dict per conversation of only USER sentences

dic = {'thankyou':"", 'thank you':"", "goodbye":"", "good bye":"", 'bye':""}


conv_only_user = {}


for file in satfiles:
    with open(file, 'r') as f:
        data =f.readlines()
        new_list =[]
        for line in data:
            # only USER input
            if "USER" in line:
                # Remove the word '[USER]'
                line = line.replace("[USER]", "")
                line = line.strip()
                new_list.append(line)
        file = file.replace('../Untitled folder/allconversations/sat\\', "")   
        file = file.replace('.txt', "")   
        for i,j in dic.items():
            new_list[-1] = new_list[-1].replace(i, j)
            new_list[-2] = new_list[-2].replace(i, j)
        conv_only_user[file] = new_list

conv_only_user2 = {}

for x, y in conv_only_user.items():
    new_dict2 = {}
    for z in y:
        new_dict2[z] = 0
    conv_only_user2[x] = new_dict2
        
# print example to check output
print(conv_only_user2['voip-00d76b791d-20130327_011609'])
    
print(conv_only_user2)   

In [4]:
#create a dict per conversation of only USER sentences

# conv_only_user = {}


# for file in satfiles:
#     with open(file, 'r') as f:
#         data =f.readlines()
#         new_dict ={}
#         for line in data:
#             # only USER input
#             if "USER" in line:
#                 # Remove the word '[USER]'
#                 line = line.replace("[USER]", "")
#                 line = line.strip()
#                 new_dict[line]= 0
#         file = file.replace('../Untitled folder/allconversations/sat\\', "")   
#         file = file.replace('.txt', "")   
#         conv_only_user[file] = new_dict

# # print example to check output
# print(conv_only_user['voip-00d76b791d-20130327_011609'])
    
# print(conv_only_user)    

In [5]:
# define the sentiment analysis function
# return a compound (or total score) between -1 and +1 per sentence. 
# -1 is the most negative, 0 is neutral and +1 is most positive.

def generatesentiment(sentence):
    # function for sentiment analysis on sentences
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sentence)
    return ss['compound']


In [6]:

# import re
# a = "hello, thankyou yes nice, bye"
# op = re.compile(r'\b\Wthankyou\W\b|\b\Wthank you\W\b|\b\Wyou\W\b|\b\Wgoodbye\W\b|\b\Wgoodbye\W\b|\b\Wbye\W\b')

# dic = {'thankyou': "", 'thank you': "", "goodbye": "", "good bye": "", 'bye': ""}

# sentence = 'thank you good bye'
# sentence2 = ' thankyou goodbye'
# a = '12'

# print(sentence)
# print(sentence2)
# for i, j in dic.items():
#         sentence = sentence.replace(i, j)
#         sentence2 = sentence2.replace(i, j)
# print(a, sentence)
# print(a, sentence2)


# if op.search(sentence)==True:
#     print('yes')
# else:
#     print('no')

In [7]:
# every conversation ends with either "thankyou" or "thank you" and "goodbye" or "good bye". 
# however, these ending sentences give a different (depending on spelling) positive score, which could influence the total
# score. 
# So, we want to get those ending sentences out:

# conv_only_user['voip-00d76b791d-20130327_011609']

# import re
# a = "hello, thank you yes nice, bye"
# op = re.compile(r'\bthankyou\b|\bthank you\b|\byou\b|\bgoodbye\b|\bgoodbye\b|\bbye\b')

# op.sub("", a)

# for conversation in conv_only_user.values():
#     total_amount_sentences = len(conversation)
#     index = 0
#     for sentence in conversation:
#         index += 1
        
#         if index == total_amount_sentences-1:
#             print(index, sentence)
                            
#         elif index == total_amount_sentences:
#             print(index, sentence)
                
# print(conv_only_user['voip-00d76b791d-20130327_011609'])
    
#print(conv_only_user) 

In [8]:
# perform sentiment analysis on every sentence

for conversation in conv_only_user2.values():
    for sentence in conversation:
        compound_sentence = generatesentiment(sentence)
        # add compound score to sentence 
        conversation[sentence] = compound_sentence
        # add total compound score of sentence
    conversation['total_score'] = sum(conversation.values())
    conversation['tot_sen_pos'] = len([x for x in conversation.values() if x > 0 and x != conversation['total_score']])
    conversation['tot_sen_neg'] = len([x for x in conversation.values() if x < 0 and x != conversation['total_score']])
        
        
 

In [9]:
# turn into dict for easy analysis

#{'moderately priced restaurant in the south part of town': 0.0, 'any': 0.0, 'great': 0.6249, 'phone number': 0.0772, 'type of food': 0.0, 'type of food is served': 0.0, 'thank you good bye': 0.6597, 'total_score': 1.3618000000000001}
#{'thai food': 0.0, 'centre': 0.0, 'yes': 0.4019, '': 0.0, 'food': 0.0, "what's the address phone number and price": 0.0772, "what's the price range": 0.0, 'thank you goodbye': 0.3612, 'total_score': 0.8403}

In [10]:
print(conv_only_user2['voip-00d76b791d-20130327_011609'])            
print(conv_only_user2['voip-0f41c16f2f-20130401_235017']) 
print(conv_only_user2['voip-e8997b10da-20130329_011626'])            
print(conv_only_user2['voip-dda7c88c6e-20130323_053057'])


{'moderately priced restaurant in the south part of town': 0.0, 'any': 0.0, 'great': 0.6249, 'phone number': 0.0772, 'type of food': 0.0, 'type of food is served': 0.0, ' ': 0.0, 'total_score': 0.7021000000000001, 'tot_sen_pos': 2, 'tot_sen_neg': 0}
{'thai food': 0.0, 'centre': 0.0, 'yes': 0.4019, '': 0.0, 'food': 0.0, "what's the address phone number and price": 0.0772, "what's the price range": 0.0, ' ': 0.0, 'total_score': 0.47909999999999997, 'tot_sen_pos': 2, 'tot_sen_neg': 0}
{'i like yes in the north part of town please': 0.7579, 'ok cheaper': 0.296, 'cheap': 0.0, 'okay what kind of food': 0.2263, 'okay can i get phone number': 0.296, 'stop phone number': -0.2263, 'okay  ': 0.2263, 'total_score': 1.5762, 'tot_sen_pos': 5, 'tot_sen_neg': 1}
{'cheap restaurant east part': 0.0, 'call number': 0.0772, 'phone there': 0.0, 'can i have the phone number': 0.0772, 'and the post code': 0.0, ' ': 0.0, 'total_score': 0.1544, 'tot_sen_pos': 2, 'tot_sen_neg': 0}
